In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# Load an image for prediction (for example, from a file)
from tensorflow.keras.preprocessing import image
import numpy as np



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Anas\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Anas\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Anas\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.

In [3]:

# Set paths to your dataset directories
train_dir = '../Datasets/Drowsy_Data/train'
test_dir = '../Datasets/Drowsy_Data/test'

# Set the image size to the input size of the model (e.g., 224x224)
img_height, img_width = 224, 224
batch_size = 32

# ImageDataGenerator for training (with data augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,  # Random shear transformation
    zoom_range=0.2,  # Random zoom transformation
    horizontal_flip=True  # Random horizontal flip
)

# ImageDataGenerator for testing (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'  # Binary classification: Drowsy (0) or Natural (1)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 5859 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.


In [32]:

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(1, activation='sigmoid')  # Output layer: sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to check the architecture
model.summary()


c:\Users\Anas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Number of batches per epoch
    epochs=10,  # You can increase this depending on your needs
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size  # Number of validation batches
)


Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 101s 549ms/step - accuracy: 0.9107 - loss: 0.2176 - val_accuracy: 0.9008 - val_loss: 0.2764
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9688 - loss: 0.0700 - val_accuracy: 0.8845 - val_loss: 0.3363
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 92s 501ms/step - accuracy: 0.9246 - loss: 0.2018 - val_accuracy: 0.9436 - val_loss: 0.1170
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9688 - loss: 0.0921 - val_accuracy: 0.9463 - val_loss: 0.1080
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 95s 521ms/step - accuracy: 0.9369 - loss: 0.1560 - val_accuracy: 0.9477 - val_loss: 0.1183
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.9688 - loss: 0.0696 - val_accuracy: 0.9470 - val_loss: 0.1272
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 97s 530ms/step - accuracy: 0.9545 - loss: 0.1185 - val_accuracy: 0.9803 - val_loss: 0.0587
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9375 - loss: 0.1200 -

In [ ]:
# Save the model
model.save('../models/drowsy_detection_model.h5')


In [36]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")


47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - accuracy: 0.9104 - loss: 0.2418
Test accuracy: 0.8988536596298218


In [ ]:
img_path = ["../Datasets/d1.jpg","../Datasets/d2.jpg","../Datasets/d3.jpg","../Datasets/d4.jpg","../Datasets/d5.jpg", "../Datasets/n1.jpg" , "../Datasets/n3.jpg" , "../Datasets/n2.jpg"]  # Replace with the image you want to classify


for i in img_path:
    print(i)
    img = image.load_img(i, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    # Predict the class
    prediction = model.predict(img_array)
    if prediction < 0.5:
        print(prediction)
        print("Drowsy")
    else:
        print(prediction)
        print("Natural")





d1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[0.6035357]]
Natural
d2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[0.03625091]]
Drowsy
d3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[0.13429569]]
Drowsy
d4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[[0.11299161]]
Drowsy
d5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
[[0.9999879]]
Natural
n1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[0.9880303]]
Natural
n3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
[[0.69839597]]
Natural
n2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[[0.0697585]]
Drowsy
